In [1]:
from urllib import request
import json
import numpy as np
import pandas as pd 

In [2]:
def get_list(keyword="country"):
    response = request.urlopen("https://nomadlist.com/api/v2/list/{}".format(keyword))
    resp_string = response.read().decode("utf-8")
    resp_dict = json.loads(resp_string)
    return resp_dict['result']

In [3]:
place_list = get_list('places')

In [4]:
def transform_cost_dict(dict_cost):
    for k,v in dict_cost.items():
        dict_cost[k] = v['USD']
    return dict_cost 

places = []
for place in place_list:
    new_dict = place['scores']
    x = transform_cost_dict(place['cost'])
    new_dict.update(x)
    new_dict["city_name"] = place["media"]["image"]["500"].split("/")[-1].split("500px")[0].strip("-")
    places.append(new_dict)

In [5]:
newdf = pd.DataFrame(places)

In [6]:
def get_corect_country(country, countries):
    for c in countries:
        if country in c:
            return c

In [7]:
newdf['country'] = newdf['city_name'].apply(lambda x: x.split("-")[-1])
country_list = get_list("countries")
newdf['country_slug'] = newdf['country'].apply(lambda x: get_corect_country(x, country_list))
newdf['country_name'] = newdf['country_slug'].apply(lambda x: x.replace('-', ' '))

In [8]:
newdf.head()

,aircon,city_name,expat,female_friendly,fragile_states_index,free_wifi_available,freedom_score,friendly_to_foreigners,leisure,lgbt_friendly,...,nomad_score,peace_score,places_to_work,press_freedom_index,racism,safety,shortTerm,country,country_slug,country_name
0,0.6,chiang-mai-thailand,620,0.8,78.8,0.6,0.2,0.6,0.62,0.6,...,1.00,0.4,1.0,44.53,0.40,0.80,599,thailand,thailand,thailand
1,1.0,bangkok-thailand,968,0.8,78.8,1.0,0.2,1.0,0.82,0.8,...,0.96,0.4,1.0,44.53,0.42,0.77,1082,thailand,thailand,thailand
2,1.0,taipei-taiwan,1498,1.0,NaN,1.0,0.6,0.8,1.00,0.8,...,0.90,NaN,1.0,24.37,0.00,1.00,1401,taiwan,taiwan,taiwan
3,1.0,melbourne-australia,2703,1.0,22.5,0.2,0.8,0.8,0.80,0.8,...,0.89,1.0,1.0,17.84,0.80,0.82,2741,australia,australia,australia
4,0.6,canggu-bali-indonesia,1283,0.8,74.9,0.6,0.2,1.0,0.90,0.6,...,0.89,0.6,0.8,41.72,0.80,0.60,951,indonesia,indonesia,indonesia


In [9]:
grouped = newdf.groupby("country_name").mean()

In [10]:
grouped.columns

Index(['aircon', 'expat', 'female_friendly', 'fragile_states_index',
       'free_wifi_available', 'freedom_score', 'friendly_to_foreigners',
       'leisure', 'lgbt_friendly', 'life_score', 'local', 'longTerm',
       'nightlife', 'nomad', 'nomadScore', 'nomad_score', 'peace_score',
       'places_to_work', 'press_freedom_index', 'racism', 'safety',
       'shortTerm'],
      dtype='object')

In [11]:
grouped.to_json('data.json')